<a href="https://colab.research.google.com/github/mahadevaasundi-origamis/Indian-ln-ocr-by-easyocr/blob/main/WebLister.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install playwright beautifulsoup4 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 14.7 MB/s eta 0:00:00


In [8]:
!playwright install

172.5 MiB [] 0% 131.3s172.5 MiB [] 0% 54.6s172.5 MiB [] 0% 29.8s172.5 MiB [] 0% 16.3s172.5 MiB [] 0% 8.0s172.5 MiB [] 1% 5.1s172.5 MiB [] 2% 4.1s172.5 MiB [] 3% 3.4s172.5 MiB [] 4% 3.0s172.5 MiB [] 5% 2.7s172.5 MiB [] 5% 3.0s172.5 MiB [] 6% 2.9s172.5 MiB [] 6% 2.8s172.5 MiB [] 7% 2.6s172.5 MiB [] 8% 2.4s172.5 MiB [] 9% 2.4s172.5 MiB [] 11% 2.2s172.5 MiB [] 12% 2.1s172.5 MiB [] 14% 2.0s172.5 MiB [] 15% 2.0s172.5 MiB [] 16% 1.8s172.5 MiB [] 17% 1.8s172.5 MiB [] 19% 1.7s172.5 MiB [] 20% 1.6s172.5 MiB [] 21% 1.6s172.5 MiB [] 22% 1.5s172.5 MiB [] 23% 1.5s172.5 MiB [] 25% 1.4s172.5 MiB [] 26% 1.4s172.5 MiB [] 26% 1.5s172.5 MiB [] 27% 1.5s172.5 MiB [] 29% 1.4s172.5 MiB [] 30% 1.3s172.5 MiB [] 31% 1.3s172.5 MiB [] 32% 1.3s172.5 MiB [] 34% 1.2s172.5 MiB [] 35% 1.2s172.5 MiB [] 37% 1.1s172.5 MiB [] 38% 1.1s172.5 MiB [] 39% 1.1s172.5 MiB [] 41% 1.0s172.5 MiB [] 42% 1.0s172.5 MiB [] 43% 1.0s172.5 MiB [] 45% 0.9s172.5 MiB [] 46% 0.9s172.5 MiB [] 48% 0.9s172.5 MiB [] 49% 0.8s172.5 MiB [] 51% 0.8s172

In [13]:
import asyncio
import requests
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime

class WebsiteURLFetcher:
    def __init__(self, target_date: str):
        """
        Initialize the fetcher with a target date in 'YYYY-MM-DD' format.
        """
        self.target_date = datetime.strptime(target_date, "%Y-%m-%d")
        self.urls = []

    def format_date_for_website(self, website: str) -> str:
        """
        Format the target date according to the website's expected date format.
        """
        if website == "Inc42":
            return self.target_date.strftime("%dth %B, %Y")  # e.g., 23rd July, 2025
        elif website == "Entrackr":
            return self.target_date.strftime("%b %d, %Y")  # e.g., Jul 24, 2025
        elif website == "VCCircle":
            return self.target_date.strftime("%d %B, %Y")  # e.g., 24 July, 2025
        elif website == "EconomicTimes":
            return self.target_date.strftime("%d %b, %Y")  # e.g., 23 Jul, 2025
        elif website == "YourStory":
            return self.target_date.strftime("%dth %b %Y")  # e.g., 24th Jul 2025
        else:
            return self.target_date.strftime("%Y-%m-%d")  # Default format

    async def fetch_inc42_urls(self):
        BASE_URL = "https://inc42.com/buzz/"
        TARGET_DATE = self.format_date_for_website("Inc42")

        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
            page = await context.new_page()
            await page.goto(BASE_URL, timeout=120_000)
            await asyncio.sleep(5)
            page_content = await page.content()
            await browser.close()

        soup = BeautifulSoup(page_content, "html.parser")
        articles = set()
        for div in soup.find_all(['div', 'article']):
            if TARGET_DATE in div.get_text():
                a = div.find('a', href=True)
                if a:
                    full_url = urljoin(BASE_URL, a['href'])
                    if full_url.startswith("https://inc42.com/buzz"):
                        articles.add(f"Inc 42 - {full_url}")
        self.urls.extend(list(articles))

    def fetch_entrackr_urls(self):
        BASE_URL = "https://entrackr.com/"
        TARGET_DATE = self.format_date_for_website("Entrackr")

        r = requests.get(BASE_URL)
        soup = BeautifulSoup(r.text, "html.parser")
        urls = set()

        for container in soup.select("div.small-post"):
            for date_span in container.find_all("span", class_="publish-date"):
                date_text = date_span.text.strip()
                if date_text.startswith(TARGET_DATE):
                    a = container.find("a", href=True)
                    if a:
                        urls.add(f"Entrackr - {urljoin(BASE_URL, a['href'])}")
                    break

        for container in soup.select("div.gallery_content"):
            for date_span in container.find_all("span", class_="publish-date"):
                date_text = date_span.text.strip()
                if date_text.startswith(TARGET_DATE):
                    a = container.find("a", href=True)
                    if a:
                        urls.add(f"Entrackr - {urljoin(BASE_URL, a['href'])}")
                    break
        self.urls.extend(list(urls))

    async def fetch_vccircle_urls(self):
        BASE_URLS = [
            "https://www.vccircle.com/deal-type/venture-capital",
            "https://www.vccircle.com/deal-type/pe",
            "https://www.vccircle.com/deal-type/m-a",
            "https://www.vccircle.com/deal-type/credit",
            "https://www.vccircle.com/industry/infrastructure",
            "https://www.vccircle.com/economy/finance",
            "https://www.vccircle.com/industry/tmt"
        ]
        TARGET_DATE = self.format_date_for_website("VCCircle")

        articles = set()
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
            page = await context.new_page()
            for BASE_URL in BASE_URLS:
                await page.goto(BASE_URL, timeout=120_000)
                await asyncio.sleep(5)
                html = await page.content()
                soup = BeautifulSoup(html, "html.parser")
                for card in soup.find_all(['article', 'div', 'section']):
                    text = card.get_text()
                    if TARGET_DATE in text:
                        a = card.find('a', href=True)
                        if a:
                            url = urljoin(BASE_URL, a['href'])
                            suffix = url.split("vccircle.com/")[-1]
                            if len(suffix) >= 15:
                                articles.add(f"VCCircle - {url}")
            await browser.close()
        self.urls.extend(list(articles))

    def fetch_economic_times_urls(self):
        BASE_URL = "https://economictimes.indiatimes.com/topic/fund-raise"
        TARGET_DATE = self.format_date_for_website("EconomicTimes")

        try:
            r = requests.get(BASE_URL, timeout=30)  # Added timeout for requests
            r.raise_for_status() # Raise an exception for bad status codes
            soup = BeautifulSoup(r.text, "html.parser")
            articles = set()
            for article in soup.find_all("div", recursive=True):
                text = article.get_text()
                if TARGET_DATE in text:
                    a = article.find("a", href=True)
                    if a and '/articleshow/' in a['href']:
                        articles.add(f"Economic Times - {urljoin(BASE_URL, a['href'])}")
            self.urls.extend(list(articles))
        except requests.exceptions.RequestException as e:
            print(f"Error fetching Economic Times URLs: {e}")


    async def fetch_yourstory_urls(self):
        BASE_URL = "https://yourstory.com"
        CATEGORY_URL = "https://yourstory.com/category/funding"
        TARGET_DATE = self.format_date_for_website("YourStory")

        async with async_playwright() as p:
            browser = await p.chromium.launch(
                headless=True,
                args=[
                    "--disable-blink-features=AutomationControlled",
                    "--no-sandbox",
                    "--disable-dev-shm-usage",
                    "--window-size=1920,1080",
                    "--disable-gpu",
                    "--no-zygote"
                ]
            )
            context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
            await context.add_init_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            page = await context.new_page()
            await page.goto(CATEGORY_URL, timeout=60000)
            await page.wait_for_selector("body", timeout=60000)
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
            await page.wait_for_timeout(5000)
            spans = await page.query_selector_all("span.sc-36431a7-0.hmxwID")
            links = set()
            for span in spans:
                date_text = (await span.inner_text()).strip()
                if date_text == TARGET_DATE:
                    container = await span.evaluate_handle("el => el.closest('div.sc-c9f6afaa-2.ckwFzl')")
                    if container:
                        a_tag = await container.query_selector("a[href^='/']")
                        if a_tag:
                            href = await a_tag.get_attribute("href")
                            if href:
                                links.add(f"YourStory - {urljoin(BASE_URL, href)}")
            await browser.close()
        self.urls.extend(list(links))

    async def fetch_all_urls(self):
        """
        Fetch URLs from all websites for the given date.
        """
        await self.fetch_inc42_urls()
        self.fetch_entrackr_urls()
        await self.fetch_vccircle_urls()
        self.fetch_economic_times_urls() # Changed to synchronous call
        await self.fetch_yourstory_urls()
        return sorted(self.urls)

    async def get_urls(self):
        """
        Run the async fetcher and return the collected URLs.
        """
        return await self.fetch_all_urls()

# Example usage:
if __name__ == "__main__":
    async def main():
        fetcher = WebsiteURLFetcher("2025-07-24")
        urls = await fetcher.get_urls()
        print("Collected URLs:")
        for url in urls:
            print(url)
    import nest_asyncio
    nest_asyncio.apply()
    await main()

Collected URLs:
Economic Times - https://economictimes.indiatimes.com/industry/services/property-/-cstruction/indiqube-leases-3-2-lakh-sq-ft-in-bengaluru-targets-aum-of-11-47-mn-sq-ft-by-fy28/articleshow/122871092.cms
Economic Times - https://economictimes.indiatimes.com/markets/bonds/despite-rbis-rate-cuts-indian-g-sec-yields-remain-range-bound-due-to-us-yield-pressures-says-rajkumar-subramanian/articleshow/122876143.cms
Economic Times - https://economictimes.indiatimes.com/markets/bonds/three-indian-infra-investment-trusts-eye-500-million-debt-in-coming-weeks-sources-say/articleshow/122886742.cms
Economic Times - https://economictimes.indiatimes.com/markets/expert-view/see-a-long-runway-for-td-segment-in-power-sector-nischal-maheshwari/articleshow/122876443.cms
Economic Times - https://economictimes.indiatimes.com/mf/analysis/iex-mutual-funds-slash-stake-while-fpis-and-retail-investors-raise-exposure-in-june-quarter/articleshow/122877836.cms
Economic Times - https://economictimes.ind